In [1]:
import os

In [2]:
os.getcwd()

'd:\\DOWNLOADS\\lays\\TaterVision--Deep-Learning-model-for-classifying-defective-and-non-defective-potato-chips\\research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'd:\\DOWNLOADS\\lays\\TaterVision--Deep-Learning-model-for-classifying-defective-and-non-defective-potato-chips'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-30 17:27:19,114: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-30 17:27:19,124: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-30 17:27:19,126: INFO: common: created directory at: artifacts]
[2023-06-30 17:27:19,129: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-30 19:05:38,606: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Server: nginx
Date: Fri, 30 Jun 2023 11:57:00 GMT
Content-Type: application/octet-stream
Content-Length: 976848132
Connection: close
Content-Disposition: attachment; filename=taterdat.zip
Accept-Ranges: none
Set-Cookie: session=7e17a51b53961992_649ec30c.XFdx24jZIEvT9-xI3OO5f0atJ58; Expires=Mon, 31-Jul-2023 11:57:00 GMT; Secure; HttpOnly; Path=/
OC-Checksum: MD5:e1409a385a13991095242b8ca1186da3
Accept-Ranges: bytes
Content-Type: application/octet-stream
Content-MD5: e1409a385a13991095242b8ca1186da3
Content-Security-Policy: default-src 'none';
X-Cont